In [1]:
%run /home/dbaciur/NTU/NTU/notes/Constants.ipynb

Constants loaded


Here we pass all our Splits through selected pretrained models to use them later in learning

In [2]:
import torch
import torchvision as tv
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import os
import math
from pathlib import Path

### Check split frames shape

In [3]:
split_frame_names = os.listdir(C.Frames.FRAMES_SCREENSHOTS_PATH)

In [4]:
frame1 = torch.load(f'{C.Frames.FRAMES_SCREENSHOTS_PATH}/{split_frame_names[0]}')
frame2 = torch.load(f'{C.Frames.FRAMES_SCREENSHOTS_PATH}/{split_frame_names[1000]}')
print(frame1.shape)
print(frame2.shape)

torch.Size([1, 720, 960, 3])
torch.Size([1, 720, 960, 3])


### Load frames df

In [5]:
frames_df = pd.read_csv(C.Frames.CHART_IMAGE_S10_LABEL_0, converters=C.F_CONVERTERS)
frames_df.head()

,Id,Video,Time,Event id,Beh id,RA,Collision events,Behs,Layout,Layouts,Splits,Origin
0,PH1104-MECHANICS_20150817_PC1__2886.1575000000...,PH1104-MECHANICS_20150817_PC1,2886.1575,729,B_WRITING_BOARD,RA13,[],[],4,"[1, 2]",[PH1104-MECHANICS_20150817_PC1__2886.157500000...,new_frames_for_chart_image_label_0_gap_10
1,PH1104-MECHANICS_20150817_PC1__44.2295__738__B...,PH1104-MECHANICS_20150817_PC1,44.2295,738,B_MOVEMENT,RA2,[],[],5,"[2, 1]",[PH1104-MECHANICS_20150817_PC1__44.2295__738__...,new_frames_for_chart_image_label_0_gap_10
2,PH1104-MECHANICS_20150817_PC1__1881.4025__751_...,PH1104-MECHANICS_20150817_PC1,1881.4025,751,B_STANDS,RA2,[],[],5,"[2, 1]",[PH1104-MECHANICS_20150817_PC1__1881.4025__751...,new_frames_for_chart_image_label_0_gap_10
3,PH1104-MECHANICS_20150817_PC1__1967.781__752__...,PH1104-MECHANICS_20150817_PC1,1967.7810,752,B_STANDS,RA2,[],[],4,"[1, 2]",[PH1104-MECHANICS_20150817_PC1__1967.781__752_...,new_frames_for_chart_image_label_0_gap_10
4,PH1104-MECHANICS_20150817_PC1__2687.4440000000...,PH1104-MECHANICS_20150817_PC1,2687.4440,763,B_WRITING_BOARD,RA2,[],[],4,"[1, 2]",[PH1104-MECHANICS_20150817_PC1__2687.444000000...,new_frames_for_chart_image_label_0_gap_10


In [6]:
def prepare_views(output_dir, frame_normalizer, nn):
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    total = len(frames_df)
    pbar = tqdm(total=total)
    for i, frame_row in frames_df.iterrows():
        pbar.update()
        
        split_names = frame_row[C.F_SPLITS]
        if split_names[0] == split_names[1]:
            split_names = [split_names[0]]
            
        for split_name in split_names:
            
            output_file = f"{output_dir}/{split_name}.pt"
            if os.path.exists(output_file):
                continue

            frame = torch.load(f"{C.Frames.FRAMES_SCREENSHOTS_PATH}/{split_name}.pt")

            # Normalizing
            normalized = frame_normalizer(frame)

            # permute to T,C,W,H and run
            nn_frame = nn.forward(normalized.permute(0,3,1,2).to('cuda')).to('cpu')
            
            torch.save(nn_frame, output_file)
            
            # clearing some memory?
            del normalized
            del nn_frame

# AlexNet

In [7]:
def normalize(vid):
    """
    im.shape = (time, height, width, channels=3)
    """
    vid = vid.float()/255
    mean = torch.Tensor([[[[0.485, 0.456, 0.406]]]]).to(vid.device)
    std = torch.Tensor([[[[0.229, 0.224, 0.225]]]]).to(vid.device)
    return (vid - mean)/std

In [8]:
def prepare_alexnet():
    al = tv.models.alexnet(pretrained=True).to('cuda')
    al.eval()
    al.classifier._modules['5'] = torch.nn.Identity()
    al.classifier._modules['6'] = torch.nn.Identity()
    return al

In [10]:
alex_net_df = prepare_views(
    output_dir=C.ALEX_NET_PATH,
    frame_normalizer= lambda frame: normalize(frame),
    nn=prepare_alexnet())

In [20]:
view1 = torch.load(f"{C.ALEX_NET_PATH}/PH1011-PHYSICS_20150922__3435.269__40__B_CHARTS_P__RA13__Left.pt") 
view2 = torch.load(f"{C.ALEX_NET_PATH}/PH1011-PHYSICS_20150922__3435.269__40__B_CHARTS_P__RA13__Right.pt")
print(view1.shape)
print(view2.shape)

torch.Size([1, 4096])
torch.Size([1, 4096])


# DeepAlexNet

In [11]:
def prepare_deep_alexnet():
    al = tv.models.alexnet(pretrained=True).to('cuda')
    al.eval()
    return al

In [12]:
deep_alex_net_df = prepare_views(
    output_dir=C.DEEP_ALEX_NET_PATH,
    frame_normalizer= lambda frame: normalize(frame),
    nn=prepare_deep_alexnet())

# ResNet

In [13]:
def prepare_resnet():
    resnet = tv.models.resnet152(pretrained=True).to('cuda')
    resnet.eval()
    resnet.fc = torch.nn.Identity()
    return resnet

In [14]:
res_net_df = prepare_views(
    output_dir=C.RES_NET_PATH,
    frame_normalizer= lambda frame: normalize(frame),
    nn=prepare_resnet())

# VGG

In [15]:
def prepare_vgg():
    vgg = tv.models.vgg19(pretrained=True).to('cuda')
    vgg.eval()
    vgg.classifier._modules['4'] = torch.nn.Identity()
    vgg.classifier._modules['5'] = torch.nn.Identity()
    vgg.classifier._modules['6'] = torch.nn.Identity()
    return vgg

In [16]:
vgg_df = prepare_views(
    output_dir=C.VGG_PATH,
    frame_normalizer= lambda frame: normalize(frame),
    nn=prepare_vgg())

# Deep VGG

In [17]:
def prepare_deep_vgg():
    vgg = tv.models.vgg19(pretrained=True).to('cuda')
    vgg.eval()
    vgg.classifier = torch.nn.Identity()
    return vgg

In [18]:
deep_vgg_df = prepare_views(
    output_dir=C.DEEP_VGG_PATH,
    frame_normalizer= lambda frame: normalize(frame),
    nn=prepare_deep_vgg())